In [1]:
import json
import jinja2
import nestedtext as nt
import os
import re
import openpyxl
AV_bench_env = jinja2.Environment(loader=jinja2.FileSystemLoader(searchpath='/projects/H3A0/workspaces/jdavid/h3v1_release/UVM_tools/uvm_analog_tb/AV_bench/templates/'))
print ("hello world")

hello world


In [2]:
#with open("dut_regs_spec.json","r") as f:
#    dut_spec = json.load(f)
dut_spec = nt.load("simple_spec.nt",top='dict')

In [3]:
if "name" in [*dut_spec]:
    print([dut_spec['name']])
    tb_name = '_'.join([dut_spec['name'],'tb'])
    print(tb_name)
try: 
    ports = dut_spec['ports']
except:
    print( "ports not in dut")
#print(ports) 
if "library" in [*dut_spec] :
    dut_library = dut_spec['library']
    tb_library = '_'.join([dut_spec['library'],'sim'])
print (tb_library)


['simple_uvm_testcase']
simple_uvm_testcase_tb
RFDV_scratch_sim


In [4]:

tb_spec = {
    'name': tb_name,
    'signals':{},
    'blocks': {
        'dut': {
            'name': dut_spec['name'],
            'ports': {},
            'library_name': dut_spec['library']
        },
        'register': {
            'name': '_'.join([dut_spec['name'],'stim','register']),
            'ports': {},
            'library_name': tb_library
        },
        'power': {
            'name': '_'.join([dut_spec['name'],'stim','power']),
            'ports': {},
            'library_name': tb_library
        },
        'bias': {
            'name': '_'.join([dut_spec['name'],'stim','bias']),
            'ports': {},
            'library_name': tb_library
        },
        'digital': {
            'name': '_'.join([dut_spec['name'],'stim','digital']),
            'ports': {},
            'library_name': tb_library
        },
        'analog': {
            'name': '_'.join([dut_spec['name'],'stim','analog']),
            'ports': {},
            'library_name': tb_library
        }
    }
}
    
print ( json.dumps(tb_spec,indent=4))

{
    "name": "simple_uvm_testcase_tb",
    "signals": {},
    "blocks": {
        "dut": {
            "name": "simple_uvm_testcase",
            "ports": {},
            "library_name": "RFDV_scratch"
        },
        "register": {
            "name": "simple_uvm_testcase_stim_register",
            "ports": {},
            "library_name": "RFDV_scratch_sim"
        },
        "power": {
            "name": "simple_uvm_testcase_stim_power",
            "ports": {},
            "library_name": "RFDV_scratch_sim"
        },
        "bias": {
            "name": "simple_uvm_testcase_stim_bias",
            "ports": {},
            "library_name": "RFDV_scratch_sim"
        },
        "digital": {
            "name": "simple_uvm_testcase_stim_digital",
            "ports": {},
            "library_name": "RFDV_scratch_sim"
        },
        "analog": {
            "name": "simple_uvm_testcase_stim_analog",
            "ports": {},
            "library_name": "RFDV_scratch_sim"
       

In [5]:

for port in ports:

    match ports[port]['direction']:
        case 'input': 
            direction = 'output'
            #tb_spec['blocks'][block]['ports'][port_name] = {'dir':'output'}
            #tb_spec['signals'][port['name']] = {'tb_block': block}
        case 'output': 
            direction = 'input'
            #tb_spec['blocks'][block]['ports'][port_name] = {'dir': 'input'}
            #tb_spec['signals'][port['name']] = {'tb_block': block}
        case other:
            direction = 'inout'
            print( " ".join([port, 'has invalid direction', ports[port]['direction']]))
    

    tb_block = ports[port]['porttype']
    if ( 'msb' in ports[port] and 'lsb' in ports[port] ):
        print ('range found ', ports[port]['msb'], port)
        tb_spec['signals'][port] = {'tb_block': tb_block,'msb':ports[port]['msb'],'lsb':ports[port]['lsb']}
        tb_spec['blocks'][tb_block]['ports'][port] = {'dir': direction,'msb':ports[port]['msb'],'lsb':ports[port]['lsb']}
        if ports[port]['direction'] == 'inputOutput':
            tb_spec['blocks']['dut']['ports'][port] = {'dir': 'inout','msb':ports[port]['msb'],'lsb':ports[port]['lsb']}                       
        else:
            tb_spec['blocks']['dut']['ports'][port] = {'dir': ports[port]['direction'],'msb':ports[port]['msb'],'lsb':ports[port]['lsb']}
    else:
        tb_spec['signals'][port] = {'tb_block': tb_block} 
        tb_spec['blocks'][tb_block]['ports'][port] = {'dir': direction}
        if ports[port]['direction'] == 'inputOutput':
            tb_spec['blocks']['dut']['ports'][port] = {'dir': 'inout'}
        else:
            tb_spec['blocks']['dut']['ports'][port] = {'dir': ports[port]['direction']}
    if tb_block == 'power':
        if ports[port].get('supply_type'):
            tb_spec['blocks'][tb_block]['ports'][port]['supply_type'] = ports[port]['supply_type']
            if ports[port]['supply_type'] == 'volts':
                if ports[port].get('supply_nom'):
                    tb_spec['blocks'][tb_block]['ports'][port]['supply_nom'] = ports[port]['supply_nom']
                else:
                    if port in ['dvdd']:
                        tb_spec['blocks'][tb_block]['ports'][port]['supply_nom'] = '0.85'
                    else: 
                        tb_spec['blocks'][tb_block]['ports'][port]['supply_nom'] = '1.0'
    if not tb_block in ['register']:
        if ports[port].get(f'{tb_block}_type'):
           tb_spec['blocks'][tb_block]['ports'][port][f'{tb_block}_type'] = ports[port][f'{tb_block}_type']
        if ports[port].get(f'{tb_block}_info'):
           tb_spec['blocks'][tb_block]['ports'][port][f'{tb_block}_info'] = ports[port][f'{tb_block}_info']
    if tb_block == 'register':
        if ports[port].get('reg_info'):
            tb_spec['blocks'][tb_block]['ports'][port]['reg_info'] = ports[port]['reg_info']
            if (not tb_spec['blocks'][tb_block].get('reg_blocks')):
                tb_spec['blocks'][tb_block]['reg_blocks'] = {}
            if ( not tb_spec['blocks'][tb_block]['reg_blocks'].get(ports[port]['reg_info']['blockid'])):
                tb_spec['blocks'][tb_block]['reg_blocks'][ports[port]['reg_info']['blockid']] = {}
            tb_spec['blocks'][tb_block]['reg_blocks'][ports[port]['reg_info']['blockid']][port] = ports[port]        
        
       


range found  3 clk_en
range found  3 dll_lock_status
range found  1 mode_sel_0
range found  1 mode_sel_1
range found  1 mode_sel_2
range found  1 mode_sel_3


In [6]:
if (dut_spec.get('tests')):
    tb_spec["tests"] = dut_spec['tests']
print( json.dumps(tb_spec, indent=4))
with open(f"{tb_spec['name']}.json", 'w') as f:
    json.dump(tb_spec, f, indent=4)

nt.dump(tb_spec, f"{tb_spec['name']}_spec.nt",sort_keys=True)

{
    "name": "simple_uvm_testcase_tb",
    "signals": {
        "AVDD1P8": {
            "tb_block": "power"
        },
        "AVSS": {
            "tb_block": "power"
        },
        "bg_enable": {
            "tb_block": "register"
        },
        "clk_en": {
            "tb_block": "register",
            "msb": "3",
            "lsb": "0"
        },
        "clk_out_0": {
            "tb_block": "digital"
        },
        "clk_out_1": {
            "tb_block": "digital"
        },
        "clk_out_2": {
            "tb_block": "digital"
        },
        "clk_out_3": {
            "tb_block": "digital"
        },
        "dll_lock_status": {
            "tb_block": "register",
            "msb": "3",
            "lsb": "0"
        },
        "mclk_in": {
            "tb_block": "digital"
        },
        "mode_sel_0": {
            "tb_block": "register",
            "msb": "1",
            "lsb": "0"
        },
        "mode_sel_1": {
            "tb_block": "registe

In [7]:
#print(os.listdir('AV_bench/templates/'))
tb_path = "simple_uvm_testcase_tb"
template = AV_bench_env.get_template('tb_top.sv')
tb_top = template.render(tb_spec)
tb_top_name = '.'.join( [tb_spec['name'],'sv'])
print (tb_top_name)
with open(f"{tb_path}/{tb_top_name}", 'w') as f:
    f.write(tb_top)
tb_regs_name = f"{tb_spec['blocks']['register']['name']}.sv"
print(tb_regs_name)
tb_regs_templ = AV_bench_env.get_template("tb_registers.sv")
tb_regs = tb_regs_templ.render(tb_spec['blocks']['register'])
with open(f"{tb_path}/{tb_regs_name}", 'w') as f:
    f.write(tb_regs) 


simple_uvm_testcase_tb.sv
simple_uvm_testcase_stim_register.sv


In [8]:
tb_spec['adrive_type'] = 'sv'
tb_power_name = f"{tb_spec['blocks']['power']['name']}.sv"
tb_power_dedrive_name = f"{tb_spec['blocks']['power']['name']}_adrive.{tb_spec['adrive_type']}"
tb_power_adrive_name = f"{tb_spec['blocks']['power']['name']}_adrive.vams"
tb_pwr_dedrive_templ = AV_bench_env.get_template(f"tb_power_adrive.sv")
tb_pwr_adrive_templ = AV_bench_env.get_template(f"tb_power_adrive.vams")
tb_pwr_adrive = tb_pwr_adrive_templ.render(tb_spec["blocks"]["power"])
with open(f"{tb_path}/{tb_power_adrive_name}","w") as f:
    f.write(tb_pwr_adrive)
tb_pwr_dedrive = tb_pwr_dedrive_templ.render(tb_spec["blocks"]["power"])
with open(f"{tb_path}/{tb_power_dedrive_name}","w") as f:
    f.write(tb_pwr_dedrive)
tb_pwr_templ = AV_bench_env.get_template('tb_power.sv')
tb_power = tb_pwr_templ.render(tb_spec["blocks"]["power"])
with open(f"{tb_path}/{tb_power_name}","w") as f:
    f.write(tb_power)

In [9]:
tb_bias_name = f"{tb_spec['blocks']['bias']['name']}.sv"
tb_bias_adrive_name = f"{tb_spec['blocks']['bias']['name']}_adrive.vams"
tb_bias_adrive_templ = AV_bench_env.get_template('tb_bias_adrive.vams')
tb_bias_adrive = tb_bias_adrive_templ.render(tb_spec["blocks"]["bias"])
with open(f"{tb_path}/{tb_bias_adrive_name}","w") as f:
    f.write(tb_bias_adrive)
tb_bias_dedrive_name = f"{tb_spec['blocks']['bias']['name']}_adrive.sv"
tb_bias_dedrive_templ = AV_bench_env.get_template('tb_bias_adrive.sv')
tb_bias_dedrive = tb_bias_dedrive_templ.render(tb_spec["blocks"]["bias"])
with open(f"{tb_path}/{tb_bias_dedrive_name}","w") as f:
    f.write(tb_bias_dedrive)
tb_bias_templ = AV_bench_env.get_template('tb_bias.sv')
tb_bias = tb_bias_templ.render(tb_spec["blocks"]["bias"])
with open(f"{tb_path}/{tb_bias_name}","w") as f:
    f.write(tb_bias)

In [10]:
tb_digital_name = f"{tb_spec['blocks']['digital']['name']}.sv"
tb_digital_templ = AV_bench_env.get_template('tb_digital.sv')
tb_digital = tb_digital_templ.render(tb_spec["blocks"]["digital"])
with open(f"{tb_path}/{tb_digital_name}","w") as f:
    f.write(tb_digital)

In [11]:
tb_analog_name = f"{tb_spec['blocks']['analog']['name']}.sv"
tb_analog_adrive_name = f"{tb_spec['blocks']['analog']['name']}_adrive.vams"
tb_analog_adrive_templ = AV_bench_env.get_template('tb_bias_adrive.vams')
tb_analog_adrive = tb_analog_adrive_templ.render(tb_spec["blocks"]["analog"])
with open(f"{tb_path}/{tb_analog_adrive_name}","w") as f:
    f.write(tb_analog_adrive)
tb_analog_dedrive_name = f"{tb_spec['blocks']['analog']['name']}_adrive.sv"
tb_analog_dedrive_templ = AV_bench_env.get_template('tb_analog_adrive.sv')
tb_analog_dedrive = tb_analog_dedrive_templ.render(tb_spec["blocks"]["analog"])
with open(f"{tb_path}/{tb_analog_dedrive_name}","w") as f:
    f.write(tb_analog_dedrive)
tb_analog_templ = AV_bench_env.get_template('tb_analog.sv')
tb_analog = tb_analog_templ.render(tb_spec["blocks"]["analog"])
with open(f"{tb_path}/{tb_analog_name}","w") as f:
    f.write(tb_analog)

In [12]:
top_analog_seq_lib_name = f"{tb_spec['blocks']['dut']['name']}_analog_seq_lib"
os.makedirs(f"{tb_path}/{top_analog_seq_lib_name}", exist_ok=True)
top_analog_seq_lib_files_name = f"{tb_path}/{top_analog_seq_lib_name}/{top_analog_seq_lib_name}_files.F"
top_analog_seq_lib_pkg_name = f"{tb_path}/{top_analog_seq_lib_name}/{top_analog_seq_lib_name}_pkg.sv"
top_analog_seq_lib_files_templ = AV_bench_env.get_template('autb_analog_seq_lib_files.F')
top_analog_seq_lib_files = top_analog_seq_lib_files_templ.render(tb_spec)
with open(f"{top_analog_seq_lib_files_name}","w") as f:
    f.write(top_analog_seq_lib_files)
top_analog_seq_lib_pkg_templ = AV_bench_env.get_template('autb_analog_seq_lib_pkg.sv')
top_analog_seq_lib_pkg = top_analog_seq_lib_pkg_templ.render(tb_spec)
with open(f"{top_analog_seq_lib_pkg_name}","w") as f:
    f.write(top_analog_seq_lib_pkg)


In [13]:
top_bias_seq_lib_name = f"{tb_spec['blocks']['dut']['name']}_bias_seq_lib"
os.makedirs(f"{tb_path}/{top_bias_seq_lib_name}", exist_ok=True)
top_bias_seq_lib_files_name = f"{tb_path}/{top_bias_seq_lib_name}/{top_bias_seq_lib_name}_files.F"
top_bias_seq_lib_pkg_name = f"{tb_path}/{top_bias_seq_lib_name}/{top_bias_seq_lib_name}_pkg.sv"
top_bias_seq_lib_files_templ = AV_bench_env.get_template('autb_bias_seq_lib_files.F')
top_bias_seq_lib_files = top_bias_seq_lib_files_templ.render(tb_spec)
with open(f"{top_bias_seq_lib_files_name}","w") as f:
    f.write(top_bias_seq_lib_files)
top_bias_seq_lib_pkg_templ = AV_bench_env.get_template('autb_bias_seq_lib_pkg.sv')
top_bias_seq_lib_pkg = top_bias_seq_lib_pkg_templ.render(tb_spec)
with open(f"{top_bias_seq_lib_pkg_name}","w") as f:
    f.write(top_bias_seq_lib_pkg)

In [14]:
top_power_seq_lib_name = f"{tb_spec['blocks']['dut']['name']}_power_seq_lib"
os.makedirs(f"{tb_path}/{top_power_seq_lib_name}", exist_ok=True)
top_power_seq_lib_files_name = f"{tb_path}/{top_power_seq_lib_name}/{top_power_seq_lib_name}_files.F"
top_power_seq_lib_pkg_name = f"{tb_path}/{top_power_seq_lib_name}/{top_power_seq_lib_name}_pkg.sv"
top_power_seq_lib_files_templ = AV_bench_env.get_template('autb_power_seq_lib_files.F')
top_power_seq_lib_files = top_power_seq_lib_files_templ.render(tb_spec)
with open(f"{top_power_seq_lib_files_name}","w") as f:
    f.write(top_power_seq_lib_files)
top_power_seq_lib_pkg_templ = AV_bench_env.get_template('autb_power_seq_lib_pkg.sv')
top_power_seq_lib_pkg = top_power_seq_lib_pkg_templ.render(tb_spec)
with open(f"{top_power_seq_lib_pkg_name}","w") as f:
    f.write(top_power_seq_lib_pkg)

In [15]:
top_digital_seq_lib_name = f"{tb_spec['blocks']['dut']['name']}_digital_seq_lib"
os.makedirs(f"{tb_path}/{top_digital_seq_lib_name}", exist_ok=True)
top_digital_seq_lib_files_name = f"{tb_path}/{top_digital_seq_lib_name}/{top_digital_seq_lib_name}_files.F"
top_digital_seq_lib_pkg_name = f"{tb_path}/{top_digital_seq_lib_name}/{top_digital_seq_lib_name}_pkg.sv"
top_digital_seq_lib_files_templ = AV_bench_env.get_template('autb_digital_seq_lib_files.F')
top_digital_seq_lib_files = top_digital_seq_lib_files_templ.render(tb_spec)
with open(f"{top_digital_seq_lib_files_name}","w") as f:
    f.write(top_digital_seq_lib_files)
top_digital_seq_lib_pkg_templ = AV_bench_env.get_template('autb_digital_seq_lib_pkg.sv')
top_digital_seq_lib_pkg = top_digital_seq_lib_pkg_templ.render(tb_spec)
with open(f"{top_digital_seq_lib_pkg_name}","w") as f:
    f.write(top_digital_seq_lib_pkg)

In [16]:
top_test_name = f"{tb_spec['blocks']['dut']['name']}_test"
os.makedirs(f"{tb_path}/{top_test_name}", exist_ok=True)
top_test_files_name = f"{tb_path}/{top_test_name}/{top_test_name}_files.F"
top_test_pkg_name = f"{tb_path}/{top_test_name}/{top_test_name}_pkg.sv"
sanity_test_name = f"{tb_path}/{top_test_name}/sanity_test.svh"
top_test_base_name = f"{tb_path}/{top_test_name}/{top_test_name}_base.svh"
top_test_files_templ = AV_bench_env.get_template('autb_test_lib_files.F')
top_test_files = top_test_files_templ.render(tb_spec)
with open(f"{top_test_files_name}","w") as f:
    f.write(top_test_files)
top_test_pkg_templ = AV_bench_env.get_template('autb_test_lib_pkg.sv')
top_test_pkg = top_test_pkg_templ.render(tb_spec)
with open(f"{top_test_pkg_name}","w") as f:
    f.write(top_test_pkg)
top_test_base_templ = AV_bench_env.get_template('autb_test_base.svh')
top_test_base = top_test_base_templ.render(tb_spec)
with open(f"{top_test_base_name}","w") as f:
    f.write(top_test_base)
sanity_test_templ = AV_bench_env.get_template('autb_sanity_test.svh')
sanity_test = sanity_test_templ.render(tb_spec)
with open(f"{sanity_test_name}","w") as f:
    f.write(sanity_test)


In [17]:
top_env_name = f"{tb_spec['blocks']['dut']['name']}_env"
os.makedirs(f"{tb_path}/{top_env_name}", exist_ok=True)
top_env_files_name = f"{tb_path}/{top_env_name}/{top_env_name}_files.F"
top_env_pkg_name = f"{tb_path}/{top_env_name}/{top_env_name}_pkg.sv"
top_env_files_templ = AV_bench_env.get_template('autb_top_env_files.F')
top_env_files = top_env_files_templ.render(tb_spec)
with open(f"{top_env_files_name}","w") as f:
    f.write(top_env_files)
top_env_pkg_templ = AV_bench_env.get_template('autb_top_env_pkg.sv')
top_env_pkg = top_env_pkg_templ.render(tb_spec)
with open(f"{top_env_pkg_name}","w") as f:
    f.write(top_env_pkg)
top_env_vseq_templ = AV_bench_env.get_template('autb_top_env_vsequencer.svh')
top_env_vseq = top_env_vseq_templ.render(tb_spec)
with open(f"{tb_path}/{top_env_name}/{tb_spec['blocks']['dut']['name']}_vsequencer.svh","w") as f:
    f.write(top_env_vseq)
top_env_templ = AV_bench_env.get_template('autb_top_env.svh')
top_env = top_env_templ.render(tb_spec)
with open(f"{tb_path}/{top_env_name}/{top_env_name}.svh","w") as f:
    f.write(top_env)
top_env_scrbrd_templ = AV_bench_env.get_template('autb_top_scoreboard.svh')
top_env_scrbrd = top_env_scrbrd_templ.render(tb_spec)
with open(f"{tb_path}/{top_env_name}/{tb_spec['blocks']['dut']['name']}_scoreboard.svh","w") as f:
    f.write(top_env_scrbrd)
top_env_config_templ = AV_bench_env.get_template('autb_top_env_config.svh')
top_env_config = top_env_config_templ.render(tb_spec)
with open(f"{tb_path}/{top_env_name}/{top_env_name}_config.svh","w") as f:
    f.write(top_env_config)



In [18]:
top_uvm_files_name = f"{tb_spec['blocks']['dut']['name']}_uvm_files.F"
top_uvm_files_templ = AV_bench_env.get_template('autb_uvm_files.F')
top_uvm_files = top_uvm_files_templ.render(tb_spec)
with open(f"{top_uvm_files_name}","w") as f:
    f.write(top_uvm_files)
top_tb_files_name = f"{tb_spec['blocks']['dut']['name']}_tb_files.F"
top_tb_files_templ = AV_bench_env.get_template('autb_tb_files.F')
top_tb_files = top_tb_files_templ.render(tb_spec)
with open(f"{tb_path}/{top_tb_files_name}","w") as f:
    f.write(top_tb_files)

In [19]:
top_test_seq_lib_name = f"{tb_spec['blocks']['dut']['name']}_test_seq_lib"
os.makedirs(f"{tb_path}/{top_test_seq_lib_name}", exist_ok=True)
top_test_seq_lib_files_name = f"{tb_path}/{top_test_seq_lib_name}/{top_test_seq_lib_name}_files.F"
top_test_seq_lib_pkg_name = f"{tb_path}/{top_test_seq_lib_name}/{top_test_seq_lib_name}_pkg.sv"
top_test_seq_lib_files_templ = AV_bench_env.get_template('autb_test_seq_lib_files.F')
top_test_seq_lib_files = top_test_seq_lib_files_templ.render(tb_spec)
with open(f"{top_test_seq_lib_files_name}","w") as f:
    f.write(top_test_seq_lib_files)
top_test_seq_lib_pkg_templ = AV_bench_env.get_template('autb_test_seq_lib_pkg.sv')
top_test_seq_lib_pkg = top_test_seq_lib_pkg_templ.render(tb_spec)
with open(f"{top_test_seq_lib_pkg_name}","w") as f:
    f.write(top_test_seq_lib_pkg)

In [20]:
top_register_seq_lib_name = f"{tb_spec['blocks']['dut']['name']}_register_seq_lib"
os.makedirs(f"{tb_path}/{top_register_seq_lib_name}", exist_ok=True)
top_register_seq_lib_files_name = f"{tb_path}/{top_register_seq_lib_name}/{top_register_seq_lib_name}_files.F"
top_register_seq_lib_pkg_name = f"{tb_path}/{top_register_seq_lib_name}/{top_register_seq_lib_name}_pkg.sv"
top_register_seq_lib_files_templ = AV_bench_env.get_template('autb_register_seq_lib_files.F')
top_register_seq_lib_files = top_register_seq_lib_files_templ.render(tb_spec)
with open(f"{top_register_seq_lib_files_name}","w") as f:
    f.write(top_register_seq_lib_files)
top_register_seq_lib_pkg_templ = AV_bench_env.get_template('autb_register_seq_lib_pkg.sv')
top_register_seq_lib_pkg = top_register_seq_lib_pkg_templ.render(tb_spec)
with open(f"{top_register_seq_lib_pkg_name}","w") as f:
    f.write(top_register_seq_lib_pkg)
def_seq_name = "register_defaults_seq.svh"
def_seq_templ = AV_bench_env.get_template('defaults_seq.sv')
def_seq = def_seq_templ.render(tb_spec)
with open(f"{tb_path}/{top_register_seq_lib_name}/{def_seq_name}","w") as f:
    f.write(def_seq)
with open(f"{tb_path}/{top_register_seq_lib_name}/{def_seq_name}","w") as f:
    f.write(def_seq)
for blockid in tb_spec['blocks']['register']['reg_blocks'].keys():
    register_on_seq_templ = AV_bench_env.get_template('block_on_seq.sv')
    on_seq = register_on_seq_templ.render(block_name = blockid, block_regs = tb_spec['blocks']['register']['reg_blocks'][blockid])
    with open(f"{tb_path}/{top_register_seq_lib_name}/{blockid}_on_seq.svh","w") as f:
        f.write(on_seq)
